In [150]:
!python -m pip install --upgrade --user ortools
!pip install  ortools

Defaulting to user installation because normal site-packages is not writeable


# Part 1: Mathematical Programming

Below `CloudResourceAllocation` class model the cloud resource allocation dataset that contains: 
> N = number of jobs,
>
> Q1 = CPU capacity,
>
> Q2 = memory capacity AND
>
> a list of cloud resources **jobs** that contains these columns: 
> > **ID, CPU demand, memory demand,and payment of a job.**

The goal of the resource allocation problem is to decide which job to accept (and which to decline), so that the CPU and memory capacity is not exceeded by the accepted jobs, and the total charged payment is maximised

In [151]:
from distutils.command.build_scripts import first_line_re
from tkinter.tix import COLUMN
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
from ortools.sat.python import cp_model
# from webob import second


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions.
    https://developers.google.com/optimization/cp/channeling
    
    """

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count



class CloudResourceAllocation:
    '''For each instance, we have following fields: 
            the 1st line of the csv files contains the number of jobs N,
            2nd line contains the CPU capacity Q1, and memory capacity Q2, 
            3rd line onwards contains the 
                    ID, CPU demand, memory demand,
                    and payment of a job.
    '''
        
    # main constructor 
    def __init__(self, N, Q1, Q2, jobs):
        '''N is the number of jobs, i.e. len(jobs)'''
        self.N = N
        self.Q1 = Q1
        self.Q2 = Q2
        self.jobs = jobs
        
    @classmethod
    def constructFromFile(cls, filePath):
        '''Read from file and construct an instance of CloudResourceAllocation'''
        with open(filePath, 'r') as file:
            first_line = file.readline()
            second_line = file.readline()
            N = int(first_line.split(',')[0])
            Q1,Q2 = int(second_line.split(',')[0]),int(second_line.split(',')[1])
            
        jobs = pd.read_csv(filePath, skiprows=range(2), header=None)
        jobs.columns = ['ID', 'CPUDemand', 'MemoryDemand','payment']
        return cls(N, Q1, Q2, jobs)
    
    def define_maths_models(self):
        '''For defining the mathematical models.
        Maximize the total payment counted by the selected jobs denoated as CiXi where the job i is charged a payment of Ci, subject to the following constraints:
            1. The selected accepted jobs' CPU demand must be less than or equal to the CPU capacity Q1.
            2. The selected accepted jobs' memory demand must be less than or equal to the memory capacity Q2.
            3. xi is binary, i.e. 0 or 1. if xi = 1, then the job is selected.
            4. i : 1 to N, i.e. the i-th job is selected if xi = 1.
        
        '''
        # self.solver = pywraplp.Solver('SolveAssignmentProblemMIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        # self.solver = pywraplp.Solver.CreateSolver('SCIP')
        self.solver = cp_model.CpModel()
        if not self.solver:
            return
        # define variables
        # x for xi, c for Ci, d1 for di1, d2 for di2
        # where i is the job ID 
        #  Ci is the payment of the job 
        #  xi is the binary variable
        # di1 is the CPU demand of the job
        # di2 is the memory demand of the job
        self.x, self.c, self.d1, self.d2 = {},{},{},{}
        for i in range(self.N):
            self.x[i] = self.solver.NewBoolVar('x[%i]' % i)
            self.c[i] = self.solver.NewIntVar(self.jobs['payment'][i],self.jobs['payment'][i], 'c[%i]' % i)
            self.d1[i] = self.solver.NewIntVar(self.jobs['CPUDemand'][i],self.jobs['CPUDemand'][i], 'd1[%i]' % i)
            self.d2[i] = self.solver.NewIntVar(self.jobs['MemoryDemand'][i],self.jobs['MemoryDemand'][i], 'd2[%i]' % i)
            
        print(f"self.x: {self.x} \n self.c payment: {self.c} \n self.d1 cpu demand: {self.d1} \n self.d2 mem demand: {self.d2}")
        # print(f"number of variables: {self.solver.NumVariables()}")
        
        # # define constraints
        # # 1. The selected accepted jobs' CPU demand must be less than or equal to the CPU capacity Q1. 
        # constraint_expr1 = [self.d1[i] for i in range(self.N)]
        # self.solver.Add(sum(constraint_expr1) <= self.Q1, 'cpu capacity constraint')
        # # 2. The selected accepted jobs' memory demand must be less than or equal to the memory capacity Q2. can be written as: d2[i]x[i] <= Q2 where x[i] is the binary variable for denoting the job i is selected or not.
        # constraint_expr2 = [self.d2[i] for i in range(self.N)]
        # self.solver.Add(sum(constraint_expr2) <= self.Q2, 'memory capacity constraint')
        
        # # define objective function
        # obj_express = [self.c[i] for i in range(self.N)]
        # self.solver.Maximize(self.solver.Sum(obj_express))
        
        constraint_expr1,constraint_expr2, obj_express = [],[],[]
        for i in range(self.N):
            constraint_expr1.append( self.d1[i] )
            constraint_expr2.append( self.d2[i] )
            # define objective function (maximize the total payment
            
            obj_express.append(self.c[i]+self.x[i])
            
        self.solver.Add(sum(constraint_expr1) <= self.Q1).OnlyEnforceIf(self.x[i])
        self.solver.Add(sum(constraint_expr2) <= self.Q2).OnlyEnforceIf(self.x[i])
        self.solver.Maximize(sum(obj_express))
            
        # constraint_expr = [ x[j] for j in range(data['num_vars'])]
        # solver.Add(sum(constraint_expr) <= data['bounds'][i])
    
    def solve_assignment_problem(self):
        '''Solve the assignment problem'''
        # Create a solver and solve with a fixed search.
        solver_temp = cp_model.CpSolver()

        # Force the solver to follow the decision strategy exactly.
        solver_temp.parameters.search_branching = cp_model.FIXED_SEARCH
        # Enumerate all solutions.
        solver_temp.parameters.enumerate_all_solutions = True
        
        # Search and print out all solutions.
        # for i in range(self.N):
        #     x 
        # solution_printer = VarArraySolutionPrinter([x, y, b] =) 
        # solver_temp.Solve(self.solver, solution_printer)
        
        # # Solve the problem.
        status = solver_temp.Solve(self.solver)
        # Print solution.
        print(f'Status = {solver_temp.StatusName(status)}')
        if status == cp_model.INFEASIBLE:
            # print infeasible boolean variables index
            print('SufficientAssumptionsForInfeasibility = 'f'{solver_temp.SufficientAssumptionsForInfeasibility()}')
            
            # print infeasible boolean variables
            infeasibles = solver_temp.SufficientAssumptionsForInfeasibility()
            for i in infeasibles:
                print('Infeasible constraint: %d' % self.solver.GetBoolVarFromProtoIndex(i))
                
        else:
            print('Objective value = %d' % solver_temp.ObjectiveValue())
            print('Solution count = %d' % solver_temp.BestObjectiveBound())
            # print(f" = {solver_temp.SearchForAllSolutions(model=self.solver,callback=)}")
            
                
        
        
        
        
        # status = self.solver.Solve()
        # if status == pywraplp.Solver.OPTIMAL:
        #     print('Objective value =', self.solver.Objective().Value())
        #     for j in range(self.N):
        #         print(self.x[j].name(), ' = ', self.x[j].solution_value())
        #     print()
        #     print('Problem solved in %f milliseconds' % self.solver.wall_time())
        #     print('Problem solved in %d iterations' % self.solver.iterations())
        #     print('Problem solved in %d branch-and-bound nodes' % self.solver.nodes())
        # else:
        #     print('The problem does not have an optimal solution.')

    
    def get_jobs(self):
        return self.jobs
    
    def __str__(self) -> str:
        return f'N: {self.N}, \nCPU Capacity Q1: {self.Q1}, Memort Capacity Q2: {self.Q2}, \n Jobs left:\n{self.jobs}'
    
    
smallFilePath = '../cloud_resource_allocation/small.csv'
largeFilePath = '../cloud_resource_allocation/large.csv'
smallDS = CloudResourceAllocation.constructFromFile(smallFilePath)
largeDS = CloudResourceAllocation.constructFromFile(largeFilePath)
smallDS.define_maths_models()

self.x: {0: x[0](0..1), 1: x[1](0..1), 2: x[2](0..1), 3: x[3](0..1), 4: x[4](0..1), 5: x[5](0..1), 6: x[6](0..1), 7: x[7](0..1), 8: x[8](0..1), 9: x[9](0..1)} 
 self.c payment: {0: c[0](836), 1: c[1](607), 2: c[2](724), 3: c[3](269), 4: c[4](381), 5: c[5](24), 6: c[6](675), 7: c[7](907), 8: c[8](962), 9: c[9](531)} 
 self.d1 cpu demand: {0: d1[0](158), 1: d1[1](136), 2: d1[2](114), 3: d1[3](203), 4: d1[4](8), 5: d1[5](183), 6: d1[6](66), 7: d1[7](41), 8: d1[8](144), 9: d1[9](36)} 
 self.d2 mem demand: {0: d2[0](317), 1: d2[1](280), 2: d2[2](61), 3: d2[3](372), 4: d2[4](11), 5: d2[5](189), 6: d2[6](66), 7: d2[7](391), 8: d2[8](377), 9: d2[9](297)}


In [152]:
smallDS.solve_assignment_problem()

Status = OPTIMAL
Objective value = 5925
Solution count = 5925


In [153]:

# large = CloudResourceAllocation(largeFilePath)
print(smallDS.__str__())
print('-'*70)
print(largeDS.__str__())

N: 10, 
CPU Capacity Q1: 1000, Memort Capacity Q2: 2000, 
 Jobs left:
   ID  CPUDemand  MemoryDemand  payment
0   1        158           317      836
1   2        136           280      607
2   3        114            61      724
3   4        203           372      269
4   5          8            11      381
5   6        183           189       24
6   7         66            66      675
7   8         41           391      907
8   9        144           377      962
9  10         36           297      531
----------------------------------------------------------------------
N: 1000, 
CPU Capacity Q1: 10000, Memort Capacity Q2: 10000, 
 Jobs left:
       ID  CPUDemand  MemoryDemand  payment
0       1       1573          1581      836
1       2       1359          1397      607
2       3       1131           304      724
3       4       2025          1857      269
4       5         71            51      381
..    ...        ...           ...      ...
995   996        592           338   

# Part 1 
The goal of the resource allocation problem is to decide which job to accept (and which to decline), so that the CPU and memory capacity is not exceeded by the accepted jobs, and the total charged payment is maximised

Below defines the **Bounding method**
> Bounding: This is to find the upper/lower bound of the optimal solution of a branch/sub-problem based on optimistic estimate.

Relax the integer constraints of the $x_i$ so that the variables can take continous values. It refers to the cpu and memory capacity can take continous values.


> Objective is to maximise `payment`
> constraints are 


In [154]:
len(smallDS.get_jobs())
smallDS.get_jobs().iloc[1:]['CPUDemand'] 
smallDS.get_jobs().iloc[0:]

,ID,CPUDemand,MemoryDemand,payment
0,1,158,317,836
1,2,136,280,607
2,3,114,61,724
3,4,203,372,269
4,5,8,11,381
5,6,183,189,24
6,7,66,66,675
7,8,41,391,907
8,9,144,377,962
9,10,36,297,531


In [155]:
# branch and bound 
import fractions


def bounding(ds:CloudResourceAllocation):
    bound = 0
    
    # payments, weights, q1_cpu_capacity, q2_memory_capacity 
    remaining_q1_cpu_capacity ,remaining_q2_memory_capacity = ds.Q1,ds.Q2
    
    # define the efficiency by adding payment per cpuDemand and payment per memoryDemand
    efficiency = [ds.get_jobs().iloc[i]['payment'] / ds.get_jobs().iloc[i]['CPUDemand'] 
                    + ds.get_jobs().iloc[i]['payment'] / ds.get_jobs().iloc[i]['MemoryDemand'] for i in range(len(ds.get_jobs()))]
    sorted_idx = sorted(range(len(efficiency)), reverse=True, key=efficiency.__getitem__)

    for i in sorted_idx:
        q1_exceed = ds.get_jobs().iloc[i]['CPUDemand'] > remaining_q1_cpu_capacity
        q2_exceed = ds.get_jobs().iloc[i]['MemoryDemand'] >remaining_q2_memory_capacity 
        if q1_exceed or q2_exceed :
            # fraction of the job that can be allocated
            # fraction = min(remaining_q1_cpu_capacity / ds.get_jobs().iloc[i]['CPUDemand'],
            #                 remaining_q2_memory_capacity / ds.get_jobs().iloc[i]['MemoryDemand'])
            fraction = remaining_q1_cpu_capacity / ds.get_jobs().iloc[i]['CPUDemand'] if q1_exceed else remaining_q2_memory_capacity / ds.get_jobs().iloc[i]['MemoryDemand']

            frac_value = ds.get_jobs().iloc[i]['payment'] * fraction
            bound += frac_value
            return bound
            
        bound += ds.get_jobs().iloc[i]['payment']
        remaining_q1_cpu_capacity -= ds.get_jobs().iloc[i]['CPUDemand']
        remaining_q2_memory_capacity -= ds.get_jobs().iloc[i]['MemoryDemand']
    return bound

In [156]:
# len(large.get_features())
# values = [1,0,0]
# values[1:]


In [157]:
# Import deque for the stack structure, copy for deep copy nodes
from collections import deque
import copy

def cloudResourceAllocation_bb_dfs(ds:CloudResourceAllocation):#(values, weights, capacity):
    # payments, weights, q1_cpu_capacity, q2_memory_capacity 
    remaining_q1_cpu_capacity ,remaining_q2_memory_capacity = ds.Q1,ds.Q2
    
    # Initialise the root, where 'expanded_item' indicates the item to be expanded at this node
    root = {
        'solution': [0] * len(ds.get_jobs()),
        'total payment': 0,
        'total cpu used': 0,
        'total memory used': 0,
        'expanded_item': 0
    }
    
    # Initially, the fringe contains the root node only
    best_solution = root
    fringe = deque()
    fringe.append(root)
    
    while len(fringe) > 0:
        # Depth-first-search, Last-In-First-Out of the stack
        node = fringe.pop()
        
        # Check if the node is a leaf node
        if node['expanded_item'] == len(ds.get_jobs()):
            if node['total payment'] > best_solution['total payment']:
                best_solution = node
                continue
        
        # Obtain the sub-problem: values, weights, capacity
        node_sub_jobs = ds.get_jobs().iloc[node['expanded_item']:]
        node_sub_q1_cpu_capacity = ds.Q1 - node['total cpu used']
        node_sub_q2_mem_capacity = ds.Q2 - node['total memory used']
        
        # Bounding on the sub-problem, and then add the value of the current solution
        bound = node['total payment'] + bounding(
            CloudResourceAllocation(
                len(node_sub_jobs),
                node_sub_q1_cpu_capacity,
                node_sub_q2_mem_capacity,
                node_sub_jobs)
        )
        # Prune the branch
        if bound <= best_solution['total payment']:
            continue
            
        # Branching on the expanded item, 0 or 1
        expanded_item = node['expanded_item']
        
        # Child 1: unselect the expanded item
        child1 = copy.deepcopy(node)
        child1['solution'][expanded_item] = 0
        child1['expanded_item'] = expanded_item + 1
        fringe.append(child1)
        
        # Child 2: select the expanded item if the capacity is enough
        new_cpu_demand = node['total cpu used']+ds.get_jobs().iloc[expanded_item]['CPUDemand']
        new_mem_demand = node['total memory used']+ds.get_jobs().iloc[expanded_item]['MemoryDemand']
        
        if new_cpu_demand <= ds.Q1 and new_mem_demand <= ds.Q2:
            child2 = copy.deepcopy(node)
            child2['solution'][expanded_item] = 1
            child2['total payment'] = node['total payment']+ ds.get_jobs().iloc[expanded_item]['payment']
            child2['total cpu used'] = new_cpu_demand
            child2['total memory used'] = new_mem_demand
            child2['expanded_item'] = expanded_item + 1
            fringe.append(child2)
    return best_solution


In [158]:
def printResult(ds:CloudResourceAllocation):
    for k,v in cloudResourceAllocation_bb_dfs(ds).items():
        suffix = ''
        # if k contains cpu
        if k.find('cpu') != -1:
            suffix = f" out of {ds.Q1}"
        if k.find('mem') != -1:
            suffix = f" out of {ds.Q2}"
        
        print(f"{k}: {v} {suffix}")
    print('-'*50)
    print('-'*50)
    return cloudResourceAllocation_bb_dfs(ds)

In [159]:
printResult(smallDS)

solution: [1, 1, 1, 0, 1, 1, 1, 1, 1, 1] 
total payment: 5647 
total cpu used: 886  out of 1000
total memory used: 1989  out of 2000
expanded_item: 10 
--------------------------------------------------
--------------------------------------------------


{'solution': [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 'total payment': 5647,
 'total cpu used': 886,
 'total memory used': 1989,
 'expanded_item': 10}

In [160]:
# printResult(largeDS)

## use the Google OR tools for part 1

In [161]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')
if not solver:
    print('No solver created.')
    exit(1)
# Create the variables x and y.
x = solver.NumVar(0, 1, 'x')
i = solver.NumVar(0, 2, 'i')
print('Number of variables =', solver.NumVariables())

# Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0, 2, 'ct')
ct.SetCoefficient(x, 1)
ct.SetCoefficient(y, 1)

Number of variables = 2


NameError: name 'y' is not defined

# Part 2: Greedy Heuristic
